In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from konlpy.tag import Twitter
from konlpy.tag import Kkma

In [3]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

In [4]:
unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

In [5]:
with open('stopwords.txt', 'r') as reader :
    stop_words0 = reader.readlines()
    
stop_words = stop_words0[0].split(',')

stop_words2 = []
for words in stop_words:
    stop_words2.append(words.decode('utf-8'))

In [6]:
tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

In [8]:
vect = CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

In [9]:
print classification_report(y_test, result)
print 
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.76      0.84      0.80       316
          1       0.83      0.74      0.79       327

avg / total       0.80      0.79      0.79       643


[[267  49]
 [ 84 243]]
